# Load required modules

In [ ]:
import psycopg2  # handles database connection to postgres db
import pandas.io.sql as sqlio  # we'll use this for improved query presentation

# Connect to the relational database

In [ ]:
# get from templates folder

# Normalized storage (3.NF)
The tables kunde and auftrag are connected with a foreign key and the database cares for integrity.

In [ ]:
# Select all elements from table auftrag
sql = """
SELECT * FROM auftrag
"""

dat = sqlio.read_sql_query(sql, conn)
dat

In [ ]:
# Select all elements from table kunde
sql = """
SELECT * FROM kunde
"""

dat = sqlio.read_sql_query(sql, conn)
dat

# Let us try to create an inconsistent state

In [ ]:
sql = """
PUT YOUR SQL HERE
"""

try:
    cur = conn.cursor()
    cur.execute(sql)
    conn.commit()
except Exception as e:
    print("Error deleting from table: {}".format(e))
    conn.commit()



The database accounts for the consistency.

# A simple transaction
Let us assume that our application's requirement is that a customer can only be added when also at least one order is inserted. Thus, we want to put these two INSERT-statements into a block of statements that are executed as if they were one (atomicity).

In [ ]:
# we first need to disable auto-commit in order to be able to define our own transactions containing more than one SQL statement
conn.autocommit = False  # this is default but we want to be sure

In [ ]:
insert_into_customer = "INSERT INTO kunde VALUES (2, 'Meier')"
insert_into_orders = "INSERT INTO auftrag VALUES (10, 2, 'Apfel')"


try:
    cur = conn.cursor()  # retrieving a database cursor starts the transaction
    # the following two statements are executed in an all-or-none-proposition
    
    # DEFINE THE TWO STATEMENTS AS A LOGICAL UNIT HERE
    
    conn.commit()    
except Exception as e:
    print("Error inserting into table: {}".format(e))
    conn.rollback()  # in case the transaction cannot be commit, we want to roll back to the previous state (CONSISTENCY)


# Select all elements from table auftrag and kunde (joined)
sql = """
SELECT * FROM kunde LEFT JOIN auftrag ON auftrag.auftrag_kunde_id=kunde.kunde_id
"""

dat = sqlio.read_sql_query(sql, conn)
dat


## Let us try to create an inconsistent state
The first statement is ok. The second statement tries to insert a row with an ID that is already assigned.

In [ ]:
insert_into_customer = "INSERT INTO kunde VALUES (3, 'Jansen')"
insert_into_orders = "INSERT INTO auftrag VALUES (10, 3, 'Banane')"  # NOTE: here is the error - the ID 10 is already used

try:
    cur = conn.cursor()  # retrieving a database cursor starts the transaction
    # the following two statements are executed in an all-or-none-proposition
    
    # DEFINE THE TWO STATEMENTS AS A LOGICAL UNIT HERE
    
    conn.commit()    
except Exception as e:
    print("Error inserting into table: {}".format(e))
    conn.rollback()  # in case the transaction cannot be commit, we want to roll back to the previous state (CONSISTENCY)


# Select all elements from table auftrag and kunde (joined)
sql = """
SELECT * FROM kunde LEFT JOIN auftrag ON auftrag.auftrag_kunde_id=kunde.kunde_id
"""

dat = sqlio.read_sql_query(sql, conn)
dat # the new customer "Jansen" should not be added to the database

## Let us take a look at isolation
We now add a new customer and simulate something like a network bottleneck, i.e. wait some seconds between the first and second statement. While we are waiting after the first insert, other transactions should not be aware of the insert. We'll use two notebooks for this (otherwise the database connection just blocks while being stuck in the first transaction).

In [ ]:
from time import sleep  # we will use this to start "waiting"

insert_into_customer = "INSERT INTO kunde VALUES (3, 'Jansen')"
insert_into_orders = "INSERT INTO auftrag VALUES (20, 3, 'Banane')"  # NOTE: the error is gone

try:
    cur = conn.cursor()  # retrieving a database cursor starts the transaction
    # the following two statements are executed in an all-or-none-proposition
    
    # DEFINE THE TWO STATEMENTS AS A LOGICAL UNIT HERE (WITH SLEEP)
    
    conn.commit()    
except Exception as e:
    print("Error inserting into table: {}".format(e))
    conn.rollback()  # in case the transaction cannot be commit, we want to roll back to the previous state (CONSISTENCY)


# Select all elements from table auftrag and kunde (joined)
sql = """
SELECT * FROM kunde LEFT JOIN auftrag ON auftrag.auftrag_kunde_id=kunde.kunde_id
"""

dat = sqlio.read_sql_query(sql, conn)
dat # the new customer "Jansen" should not be added to the database

### This cell is used to repeat the case

In [ ]:
try:
    cur = conn.cursor()
    cur.execute("DELETE FROM auftrag WHERE auftrag_kunde_id=3")
    cur.execute("DELETE FROM kunde WHERE kunde_id=3")    
    conn.commit()    
except Exception as e:
    print("Error deleting from table: {}".format(e))
    conn.rollback()  # in case the transaction cannot be commit, we want to roll back to the previous state (CONSISTENCY)